In [ ]:
import syft as sy

In [ ]:
worker = sy.Worker.named("test-domain-1", processes=1)
domain_client = worker.root_client

In [ ]:
results = domain_client.api.services.dataset.get_all()
results

In [ ]:
assert len(results) == 1

In [ ]:
dataset = results[-1]

In [ ]:
dataset

In [ ]:
mock = dataset.assets[0].mock
mock

In [ ]:
mock["Trade Value (US$)"].sum()

In [ ]:
@sy.syft_function(input_policy=sy.ExactMatch(trade_data=mock),
                  output_policy=sy.SingleExecutionExactOutput())
def sum_trade_value_mil(trade_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    df = trade_data
    total = df["Trade Value (US$)"].sum()
    return (float(total / 1_000_000), float(noise))

In [ ]:
result = sum_trade_value_mil(trade_data=mock)
result

In [ ]:
assert result[0] == 9.738381

In [ ]:
assert isinstance(result[1], float)

In [ ]:
assert len(sum_trade_value_mil.kwargs) == 1

In [ ]:
assert "trade_data" in sum_trade_value_mil.kwargs

In [ ]:
assert len(sum_trade_value_mil.outputs) == 0

In [ ]:
assert sum_trade_value_mil.input_policy.inputs["trade_data"] == mock.id

In [ ]:
sum_trade_value_mil.code

In [ ]:
guest_domain_client = domain_client.guest()

In [ ]:
guest_credentials = guest_domain_client.credentials
guest_credentials

In [ ]:
guest_domain_client.register(name="Jane Doe", email="jane@caltech.edu", password="abc123", institution="Caltech", website="https://www.caltech.edu/")

In [ ]:
guest_domain_client.login(email="jane@caltech.edu", password="abc123")

In [ ]:
assert guest_domain_client.credentials != guest_credentials

In [ ]:
guest_domain_client.api.services.code.request_code_execution(sum_trade_value_mil)

In [ ]:
guest_domain_client.code

In [ ]:
assert len(guest_domain_client.api.services.code.get_all()) == 1

In [ ]:
new_project = sy.Project(name="My Cool UN Project")

In [ ]:
proj_desc = """Hi, I want to calculate the trade volume in million's with my cool code."""
new_project.set_description(proj_desc)

In [ ]:
submitted_code = guest_domain_client.code[0]

In [ ]:
submitted_code

In [ ]:
new_project.add_request(obj=submitted_code, permission=sy.UserCodeStatus.EXECUTE)

In [ ]:
new_project.changes

In [ ]:
new_project

In [ ]:
guest_domain_client.submit_project(new_project)

In [ ]:
guest_domain_client._api = None
_ = guest_domain_client.api

In [ ]:
result = guest_domain_client.api.services.code.sum_trade_value_mil(trade_data=mock.id)
result

In [ ]:
assert isinstance(result, sy.SyftNotReady)